<a href="https://colab.research.google.com/github/AI-Lai/ExperimentingWithAtari/blob/main/ATARI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Dependencies

In [1]:
%pip install tensorflow
!pip install pyglet
print("done")
!pip install gym keras-rl2 gym[atari]
print("done")
%pip install -U gym>=0.21.0
print("done")
%pip install -U gym[atari,accept-rom-license]
print("DONE")
!pip install pyvirtualdisplay
!pip install reverb

done
     |████████████████████████████████| 52 kB 818 kB/s 
done
done
     |████████████████████████████████| 1.6 MB 13.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441026 sha256=d8008cb33d87bc5cda9211c00cdb267e06627bb40fddf8367f43256b1dd335c5
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license
DONE
  Created wheel for reverb: filename=reverb-2.0.1-py3-none-any.whl size=3689 sha256=2ba53929a20e452a550159b35a0395d3da5c582f893b111c53166e8566966987
  Stored in directory: /root/.cache/pip/wheels/32/f5/10/f2e2651a09c36a5d000f4cc763058488a14918292d38c0de1e
Successfully built reverb


# 1. Test Random Environment with OpenAI Gym

In [2]:
import gym 
import random

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [3]:
env = gym.make('Pong-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
episodes = 3
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:50.0
Episode:2 Score:105.0
Episode:3 Score:55.0


# 2. Create a Deep Learning Model with Keras

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [13]:
# del model
# in case of random error, rebuild the model

In [14]:
model = build_model(height, width, channels, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 6)                 1

# 3. Build Agent with Keras-RL

In [10]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy


In [17]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=1000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  # enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-2))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  746/10000: episode: 1, duration: 10.929s, episode steps: 746, steps per second:  68, episode reward: 120.000, mean reward:  0.161 [ 0.000, 30.000], mean action: 2.815 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1882/10000: episode: 2, duration: 653.752s, episode steps: 1136, steps per second:   2, episode reward: 170.000, mean reward:  0.150 [ 0.000, 30.000], mean action: 2.963 [0.000, 5.000],  loss: 0.690011, mean_q: 5.082212, mean_eps: 0.100000
 2477/10000: episode: 3, duration: 434.699s, episode steps: 595, steps per second:   1, episode reward: 80.000, mean reward:  0.134 [ 0.000, 20.000], mean action: 2.763 [0.000, 5.000],  loss: 0.162999, mean_q: 4.772885, mean_eps: 0.100000
 3191/10000: episode: 4, duration: 518.976s, episode steps: 714, steps per second:   1, episode reward: 115.000, mean reward:  0.161 [ 0.000, 30.000], mean action: 2.917 [0.000, 5.000],  loss: 0.220692, mean_q: 5.253962, mean_eps: 0.100000
 3836/10000: episode: 5, duration: 458.637s, episode steps: 645, steps per second:   1, episode reward: 125.000, mean reward:  0.194 [ 0.000, 25.000], mean action: 2.698 [0.000, 5.000],  loss: 0.159486, mean_q: 5.281538, mean_eps: 0.100000
 4659/10000: episode: 6, duration: 580.8

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')